In [2]:
import nuclio

In [3]:
%run set_env.ipynb

In [4]:
%%nuclio env 
SHARD=0
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
IGZ_CONTAINER=${IGZ_CONTAINER} 
RAW_VIDEO_STREAM=${RAW_VIDEO_STREAM}
TAGGED_VIDEO_STREAM=${TAGGED_VIDEO_STREAM}
LOWRES_VIDEO_STREAM=${LOWRES_VIDEO_STREAM} 
CAMERA_LIST_TBL=${CAMERA_LIST_TBL} 

%nuclio: setting 'SHARD' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'IGZ_CONTAINER' environment variable
%nuclio: setting 'RAW_VIDEO_STREAM' environment variable
%nuclio: setting 'TAGGED_VIDEO_STREAM' environment variable
%nuclio: setting 'LOWRES_VIDEO_STREAM' environment variable
%nuclio: setting 'CAMERA_LIST_TBL' environment variable


In [5]:
%nuclio mount /User /users/admin

mounting volume path /User as /users/admin


In [ ]:
%%nuclio cmd -c
pip install requests opencv-python

In [ ]:
# nuclio: start-code

In [2]:
import requests
import json
import base64
import os
import time

In [23]:
def seek_location(context,
                  shardId: int = 0,
                  container: str = '',
                  stream: str = "",
                  url = '',
                 headers = '',
                 secondsLapsed = 1):
    payload = {
            "ShardId": shardId,
            "Type": "TIME",
        "TimestampSec":int(time.time())-secondsLapsed,
        "TimestampNSec" : 0
          }
    headers["X-v3io-function"] = "Seek"
    r = requests.post(url, headers=headers,json=payload)
    return json.loads(r.text)['Location']

In [ ]:
def get_image( context,
              url,
              shardId,
              container,
              stream,
              secondsLapsed):
    location = seek_location(context,shardId=shardId,
                             container=container,
                             stream=stream,
                             url=url,
                            headers=headers,
                            secondsLapsed = secondsLapsed)
    img_payload = {
            "ShardId":    shardId,
            "Location":   location,
        "Limit":1
    }
    headers["X-v3io-function"] = "GetRecords"
    r = requests.post(url, headers=headers,json=img_payload)
    
    response=json.loads(r.text)
    for record in response['Records']:
        data = base64.b64decode(record['Data'])
    
    return data

In [24]:
def handler(context,event):
    params = event.fields
    shardId=int(params['shardId'])
    container=params['container']
    stream=params['stream']
    
    # Fire up the Kafka Consumer
    url = "http://v3io-webapi:8081/%s/%s/"% (container,stream)
    headers = { "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
    }

    secondsLapsed = 1
    while not data in get_image(context,url,shardId,container,stream,secondsLapsed)
          secondsLapsed += 1
    
    #img = np.asarray(bytearray(data), dtype="uint8")
    #img = cv2.imdecode(img,cv2.COLOR_BGR2RGBA)
    #cv2.imshow('Faces',img)
    #return context.Response(body=data,headers=None,content_type='image/jpeg',status_code=201)
    return data

In [25]:
# nuclio: end-code

In [26]:
%nuclio deploy -p objectrecognition -n latest-image-retrieve -c

[nuclio] 2020-07-05 16:47:05,405 (info) Build complete
[nuclio] 2020-07-05 16:47:11,463 (info) Function deploy complete
[nuclio] 2020-07-05 16:47:11,471 done updating latest-image-retrieve, function address: 3.128.161.62:32746
%nuclio: function deployed


In [3]:
event = nuclio.Event(body=b'jkk')

In [ ]:
handler(context,event)

In [4]:
event.

{}